## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Leningradskiy,69.3833,178.4167,-8.79,91,31,6.55,RU,2021-11-03 20:30:35
1,1,Aras,42.5617,-2.3560,45.01,85,40,11.50,ES,2021-11-03 20:30:36
2,2,Vysokogornyy,50.1167,139.1500,26.17,83,60,2.17,RU,2021-11-03 20:30:36
3,3,Mataura,-46.1927,168.8643,54.48,49,30,2.33,NZ,2021-11-03 20:30:36
4,4,Clyde River,70.4692,-68.5914,21.56,86,97,32.21,CA,2021-11-03 20:30:37


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation?"))
max_temp = float(input("What is the maximum temperature you would like for your vacation?"))

In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,Carnarvon,-24.8667,113.6333,64.47,82,0,16.11,AU,2021-11-03 20:28:22
9,9,Ganzhou,28.8184,115.4327,62.01,87,100,1.01,CN,2021-11-03 20:30:38
11,11,Durgapur,23.4833,87.3167,69.98,63,87,2.68,IN,2021-11-03 20:30:38
12,12,Cape Town,-33.9258,18.4232,61.27,80,40,6.91,ZA,2021-11-03 20:29:59
16,16,Tuy Hoa,13.0833,109.3000,74.28,93,100,4.07,VN,2021-11-03 20:26:55
18,18,Rikitea,-23.1203,-134.9692,73.80,73,100,14.99,PF,2021-11-03 20:30:39
20,20,Arraial Do Cabo,-22.9661,-42.0278,73.36,83,40,11.50,BR,2021-11-03 20:27:24
22,22,Castro,-24.7911,-50.0119,70.61,68,75,2.17,BR,2021-11-03 20:30:41
30,30,Mawlaik,23.6333,94.4167,69.01,97,40,1.50,MM,2021-11-03 20:30:43
37,37,Hermanus,-34.4187,19.2345,63.81,78,4,11.59,ZA,2021-11-03 20:27:11


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       145
City          145
Lat           145
Lng           145
Max Temp      145
Humidity      145
Cloudiness    145
Wind Speed    145
Country       143
Date          145
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
#print(clea)

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Carnarvon,AU,64.47,-24.8667,113.6333,
9,Ganzhou,CN,62.01,28.8184,115.4327,
11,Durgapur,IN,69.98,23.4833,87.3167,
12,Cape Town,ZA,61.27,-33.9258,18.4232,
16,Tuy Hoa,VN,74.28,13.0833,109.3000,
18,Rikitea,PF,73.80,-23.1203,-134.9692,
20,Arraial Do Cabo,BR,73.36,-22.9661,-42.0278,
22,Castro,BR,70.61,-24.7911,-50.0119,
30,Mawlaik,MM,69.01,23.6333,94.4167,
37,Hermanus,ZA,63.81,-34.4187,19.2345,


In [33]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found....skipping")
        

Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping


In [39]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [40]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>

<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))